In [1]:
import pandas as pd
from loguru import logger
# Setup environment for running MATLAB code if not done externally
import os
%load_ext autoreload

# os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"
os.environ["MR"] = f"{os.environ['HOME']}/MATLAB/R2023b"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/runtime/glnxa64:{MR}/bin/glnxa64:{MR}/sys/os/glnxa64:{MR}/sys/opengl/lib/glnxa64"


In [2]:
# Visualization packages

from pathlib import Path
import os
import hjson
import json
import numpy as np
import pandas as pd
from loguru import logger
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

resample_figures = False

# Visualize result so far
with open( Path("data") / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)

# Load plot configuration
with open( Path("data") / "plot_config.hjson") as f:
    plt_config = hjson.load(f)

In [3]:
df = pd.read_csv('data/dev.csv', parse_dates=['time'], index_col='time')

In [11]:
from importlib import reload
from models_psa.solar_med import SolarMED
# reload(SolarMED)
# from models_psa.solar_med import SolarMED

%autoreload 2

idx_start = 10
idx_end = 100
df_mod = pd.DataFrame()

# Initialize model
model = SolarMED(
    sample_time=60,
    curve_fits_path='data/curve_fits.json',
    
    # Initial states. Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[idx_start], df['Tts_h_m'].iloc[idx_start], df['Tts_h_b'].iloc[idx_start]], 
    Tts_c=[df['Tts_c_t'].iloc[idx_start], df['Tts_c_m'].iloc[idx_start], df['Tts_c_b'].iloc[idx_start]],
    Tsf_in=df['Tsf_in'].iloc[idx_start],
    
    cost_w = 3, # €/m³ 
    cost_e = 0.05 # €/kWhe
)

# Run model
for idx in range(idx_start, idx_end):
    # idx = 1
    ds = df.iloc[idx]
    # df.to_csv('data/dev.csv')
        
    logger.info(f"Running iteration {idx} out of {len(df)}, last operation state was {model.operating_state}")
        
    model.step(
        # Decision variables
        ## MED
        mmed_s=ds['qmed_s'],
        mmed_f=ds['qmed_f'],
        Tmed_s_in=ds['Tmed_s_in'],
        Tmed_c_out=ds['Tmed_c_out'],
        ## Thermal storage
        mts_src=ds['qts_src'],
        ## Solar field
        Tsf_out=ds['Tsf_out'],
        
        # Inputs
        # When the solar field is starting up, a flow can be provided to sync the model with the real system, if a valid Tsf_out is provided, it will be prioritized
        msf=ds['qsf'] if ds['qsf'] > 4 else None,
        
        # Environment variables
        Tmed_c_in=ds['Tmed_c_in'],
        Tamb=ds['Tamb'],
        I=ds['I'],
    )
    
    logger.info(f'Tsf,in,mod = {model.Tsf_in:.2f}, Tsf,in,meas = {ds["Tsf_in"]:.2f}, msf_meas = {ds["qsf"]:.2f}, msf_mod = {model.msf:.2f}')
    
    df_mod = model.to_dataframe(df_mod)

2024-02-28 17:51:09.888 | INFO     | models_psa.solar_med:init_matlab_engine:891 - MATLAB engine initialized
2024-02-28 17:51:09.891 | INFO     | __main__:<module>:32 - Running iteration 10 out of 587, last operation state was SolarMED_states.IDLE
2024-02-28 17:51:09.893 | DEBUG    | models_psa.validation:within_range_or_zero_or_max:21 - (mmed_s_sp) Value 0.03 is less than 20.016 -> 0.0
2024-02-28 17:51:09.894 | DEBUG    | models_psa.validation:within_range_or_zero_or_max:21 - (mmed_f_sp) Value 0.01 is less than 5 -> 0.0
2024-02-28 17:51:09.896 | DEBUG    | models_psa.validation:within_range_or_zero_or_max:21 - (Tmed_s_in_sp) Value 17.59 is less than 60 -> 0.0
2024-02-28 17:51:09.898 | DEBUG    | models_psa.validation:within_range_or_zero_or_max:21 - (Tsf_out_sp) Value 18.70 is less than 65 -> 0.0
2024-02-28 17:51:09.899 | DEBUG    | models_psa.solar_med:step:769 - Solar field warm up mode, using provided qsf: 5.7 (m³/h) instead of Tsf,out,sp
2024-02-28 17:51:09.901 | DEBUG    | models

In [10]:
# Sync model index with measured data
df_mod.index = df.index[idx_start:idx]

fig = experimental_results_plot(plt_config, df.iloc[idx_start:idx], df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)

2024-02-28 17:05:17.918 | INFO     | phd_visualizations.test_timeseries:experimental_results_plot:239 - Optimization updates not shown in plot, show_optimization_updates: false
2024-02-28 17:05:17.919 | DEBUG    | phd_visualizations.test_timeseries:add_trace:32 - Attempting to add Tamb
2024-02-28 17:05:17.920 | DEBUG    | phd_visualizations.test_timeseries:add_trace:65 - legend_id: global_legend, legend: legend for trace T<sub>amb</sub>
2024-02-28 17:05:17.924 | INFO     | phd_visualizations.test_timeseries:add_trace:162 - Trace T<sub>amb</sub> added in yaxis1 (left), row 1, uncertainty: False, comparison: True
2024-02-28 17:05:17.928 | DEBUG    | phd_visualizations.test_timeseries:add_trace:32 - Attempting to add Tmed_c_in
2024-02-28 17:05:17.929 | DEBUG    | phd_visualizations.test_timeseries:add_trace:65 - legend_id: global_legend, legend: legend for trace T<sub>med,c,in</sub> (⁰C)
2024-02-28 17:05:17.934 | INFO     | phd_visualizations.test_timeseries:add_trace:162 - Trace T<sub>me